Step 0: Enable GPU in Runtime Settings and Set up Python Execution Paths
Click on the menu: Runtime → Change runtime type.

Under Hardware accelerator, select: GPU (for NVIDIA GPUs like T4, P100, A100 depending on your Colab plan)

Click “Save”.

Now, your notebook will run with GPU acceleration.

Next, let's verify GPU availability

In [ ]:

import torch
# If it returns True, GPU is active.
torch.cuda.is_available()

True

In [ ]:

# Use this command to see which GPU you have
# You will see the GPU version and the total availabel memory
!nvidia-smi

Mon Dec  8 22:33:01 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   43C    P8              9W /   70W |       2MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

Step 0: Install required packages and libraries

In [ ]:
!pip install pandas numpy scikit-learn matplotlib seaborn nltk
!pip install torch torchvision torchaudio transformers


In [ ]:
# All libraries
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import nltk
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from transformers import AutoTokenizer, AutoModelForCausalLM
from sklearn.model_selection import train_test_split
from sklearn.metrics import (accuracy_score, classification_report, confusion_matrix)
import kagglehub

nltk.download('stopwords')
from nltk.corpus import stopwords


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


Step 1: Download and clean the data from KaggleHub for TF-IDF

In [ ]:
# download the latest version
path = kagglehub.dataset_download("saurabhshahane/fake-news-classification")

# data_path = "/kaggle/input/fake-news-classification/WELFake_Dataset.csv"
data_path = os.path.join(path, "WELFake_Dataset.csv")

df = pd.read_csv(data_path)

# drop unused col
df = df.drop(columns=["Unnamed: 0"])

# combine the Title and text
df['content'] = df['title'].fillna('') + " " + df['text'].fillna('')
#make everything lowercase and remove spaces for TF-IDF search to happen more easily and accurately
df['content'] = df['content'].str.lower().str.replace(r'[^a-z\s]', '', regex = True)
X = df['content']
y = df['label']

# show data for reference
print(df.head())
print(df.describe())

Using Colab cache for faster access to the 'fake-news-classification' dataset.
                                               title  \
0  LAW ENFORCEMENT ON HIGH ALERT Following Threat...   
1                                                NaN   
2  UNBELIEVABLE! OBAMA’S ATTORNEY GENERAL SAYS MO...   
3  Bobby Jindal, raised Hindu, uses story of Chri...   
4  SATAN 2: Russia unvelis an image of its terrif...   

                                                text  label  \
0  No comment is expected from Barack Obama Membe...      1   
1     Did they post their votes for Hillary already?      1   
2   Now, most of the demonstrators gathered last ...      1   
3  A dozen politically active pastors came here f...      0   
4  The RS-28 Sarmat missile, dubbed Satan 2, will...      1   

                                             content  
0  law enforcement on high alert following threat...  
1      did they post their votes for hillary already  
2  unbelievable obamas attorney general 

Step 2: Split the data to train it

In [ ]:
from sklearn.model_selection import train_test_split

# Split the dataset
X_train, X_test, y_train, y_test = train_test_split(
    X,
    y,
    test_size=0.2,      # 20% test set
    random_state=42,    # ensures reproducibility
    stratify=y          # keeps label proportions the same in train/test
)

print("Training size:", len(X_train))
print("Test size:", len(X_test))

Training size: 57707
Test size: 14427


In [ ]:
from __future__ import annotations
from dataclasses import dataclass, field, asdict
from typing import Callable, Dict, List, Tuple, Optional, Any
import json, math, re, textwrap, random, os, sys
import math
from collections import Counter, defaultdict


In [ ]:
CORPUS = []

for i, rows in df.iterrows():
    title = rows["title"]
    text = rows["text"]
    label = rows["label"]

    if pd.isna(title):
        title = ""
    if pd.isna(text):
        text = ""

    CORPUS.append({
        "id": str(i),
        "title": str(title),
        "text": str(text),
        "label": int(rows["label"])
    })

In [ ]:

def tokenize(text: str) -> List[str]:
    return re.findall(r"[a-zA-Z0-9']+", text.lower())

DOC_TOKENS = [tokenize(d["title"] + " " + d["text"]) for d in CORPUS]

VOCAB = sorted(set(t for doc in DOC_TOKENS for t in doc))

def compute_tf(tokens: List[str]) -> Dict[str, float]:
    # Input: A list of all the words in a document
    # Output: A dictionary of the frequency of each word

    doc_length = len(tokens)
    term_counts = Counter(tokens)
    tf_scores = {}
    for term, count in term_counts.items():
        tf_scores[term] = count / doc_length

    return tf_scores



In [ ]:
def compute_df(doc_tokens: List[List[str]]) -> Dict[str, float]:
    # Input: A list of lists of tokens in each document
    # Output: A dictionary of the counts of each word appearing across the documents

    df_counts = defaultdict(int)

    for doc in doc_tokens:
        unique_tokens = set(doc)
        for token in unique_tokens:
            df_counts[token] += 1

    return dict(df_counts)

In [ ]:

DF = compute_df(DOC_TOKENS) # Get the DF
N_DOC = len(DOC_TOKENS) # number of docs
IDF = {t: math.log((N_DOC + 1) / (DF[t] + 0.5)) + 1 for t in VOCAB} # Inverse document frequency

def tfidf_vector(tokens: List[str]) -> Dict[str, float]:
    # Input: A list of words in a document
    # Output: A dictionary of tf-idf score of each word
    tf = compute_tf(tokens)
    vec = {t: tf[t] * IDF.get(t, 0.0) for t in tf}
    return vec

DOC_VECS = [tfidf_vector(tokens) for tokens in DOC_TOKENS]



In [ ]:
def cosine(a: Dict[str, float], b: Dict[str, float]) -> float:
    # Inputs: Two dictrionaries of tf-idf vectors of two document
    # Output: The cosine similarity scalar between the two vector

    if not a or not b:
        return 0.0

    all_terms = set(a.keys()) | set(b.keys())

    dot_product = 0.0
    norm_a_sq = 0.0
    norm_b_sq = 0.0

    for term in all_terms:
        weight_a = a.get(term, 0.0)
        weight_b = b.get(term, 0.0)

        dot_product += weight_a * weight_b
        norm_a_sq += weight_a ** 2
        norm_b_sq += weight_b ** 2

    denominator = math.sqrt(norm_a_sq) * math.sqrt(norm_b_sq)

    similarity = 0.0
    if denominator > 0:
        similarity = dot_product / denominator

    return similarity

In [ ]:
# summarizer
def summarize_title(doc_text: str) -> str:
  sentences = doc_text.split('.')
  if len(sentences) == 0:
    return doc_text
  else:
    return sentences[0].strip() + '.'

In [ ]:
def search_corpus(query: str, k: int = 3) -> List[Dict[str, Any]]:
    qvec = tfidf_vector(tokenize(query))
    scored = [(cosine(qvec, v), i) for i, v in enumerate(DOC_VECS)]
    scored.sort(reverse=True)

    top_k = scored[:k]
    max_score = top_k[0][0] if top_k else 0.0

    results = []
    for score, idx in scored[:k]:
        d = CORPUS[idx].copy()
        d["score"] = float(score)
        results.append(d)
    return {
        "results": results,
        "query": query,
        "confidence": float(max_score),
        "summary": summarize_title(query)
    }


In [ ]:
def tool_search(query: str, k: int = 3) -> Dict[str, Any]:
    hits = search_corpus(query, k=k)
    # Return a concise, citation-friendly payload
    return {
        "tool": "search",
        "query": query,
        "confidence": hits["confidence"],
        "summary": hits["summary"],
        "results": [
            {"id": h["id"], "title": h["title"], "score": h["score"], "snippet": h["text"][:240] + ("..." if len(h["text"]) > 240 else "")}
            for h in hits["results"]
        ],
    }

TOOLS = {
    "search": {
        "schema": {"query": "str", "k": "int? (default=3)"},
        "fn": tool_search
    },
    "finish": {
        "schema": {"answer": "str"},
        "fn": lambda answer: {"tool": "finish", "answer": answer}
    }
}

In [ ]:
# milestone2: prompting

In [ ]:
import re
import json

# Regex to match the action patterns
ACTION_RE = re.compile(
    r'^(search|finish)\[(.*)\]$'
)


def parse_action(line):
    """
    Parse an action line of the form:
        Action: search[query="...", k=3]
        Action: finish[answer="..."]
    OR with no 'Action:' prefix (our trajectory stores this way).

    Returns:
        {"type": <str>, "args": <dict>}   OR   None
    """

    # Remove optional prefix
    if line.startswith("Action:"):
        line = line[len("Action:"):].strip()

    m = ACTION_RE.match(line)
    if not m:
        return None

    action_type, inside = m.groups()

    # Split args inside brackets
    parts = [p.strip() for p in inside.split(",")]

    args = {}

    for p in parts:
        if "=" not in p:
            continue
        key, value = p.split("=", 1)
        key = key.strip()
        value = value.strip()

        # Strip quotes "..."
        if value.startswith('"') and value.endswith('"'):
            value = value[1:-1]

        # Convert integer if appropriate
        if value.isdigit():
            value = int(value)

        args[key] = value

    return {"type": action_type, "args": args}



In [ ]:

SYSTEM_PROMPT = """
You are a misinformation-detection agent.
You operate in a strict loop of:

    Thought: <your reasoning>
    Action: <a single tool call>

You have exactly TWO valid actions:

1. search[query="<text>", k=<int>]
   - MUST be called as:
       Action: search[query="<some text>", k=<integer>]
   - MUST include both query="..." and k=...
   - MUST NOT use lists or multiple strings.
   - VALID:
       Action: search[query="satan 2 russia missile", k=5]
       Action: search[query="obama christmas trees", k=3]
   - INVALID (NEVER DO THESE):
       search["a","b"]
       search["...", 10]
       search("something")
       Action: Action: search[...]
       Action: search[text="...", k=5]

2. finish[answer="<label>"]
   - This ENDS the interaction.
   - The answer MUST be EXACTLY one of:
       "fake"
       "true"
   - VALID:
       Action: finish[answer="fake"]
       Action: finish[answer="true"]
   - INVALID:
       Action: finish[answer="this is probably fake"]
       Action: finish[answer="No"]
       Action: finish[answer="True. This headline is real."]

TASK:
- You are given a news headline.
- Use search[...] to retrieve evidence from the corpus.
- Reason step by step in Thought: lines.
- When you are confident, choose:
    - "fake"  → the headline is misinformation / false
    - "true"  → the headline is accurate / real
- Then call EXACTLY ONE final action:
    Action: finish[answer="fake"]
    OR
    Action: finish[answer="true"]

FORMATTING RULES (MANDATORY):

- At EVERY step you MUST output TWO lines in this order:
    Thought: <1–3 sentences of reasoning>
    Action: <ONE valid action as defined above>

- NEVER output Action without a Thought line right before it.
- NEVER put the tool call inside Thought: — only inside Action:.
- NEVER output more than ONE Action per step.
- NEVER output “Action: Action: ...”.

EXAMPLE (FOLLOW THIS PATTERN):

Thought: I should search the corpus for evidence about 'SATAN 2' and Russia.
Action: search[query="satan 2 russian missile", k=3]
Observation: {...}

Thought: The documents suggest this is misinformation, so it is fake.
Action: finish[answer="fake"]

Always follow this Thought → Action pattern for each step.
"""


from dataclasses import asdict

def make_prompt(question, trajectory):
    """
    Build a ReAct-style prompt from:
      • the system instructions
      • the user question
      • the ongoing Thought → Action → Observation steps

    NOTE: We DO NOT end with 'Thought:' here. The model itself must
    generate 'Thought:' followed by its reasoning.
    """
    lines = [SYSTEM_PROMPT]
    lines.append(f"User Question: {question}\n")

    for step in trajectory:
        step = asdict(step)
        if step.get("thought"):
            lines.append(f"Thought: {step['thought']}")
        if step.get("action"):
            # 'action' already includes "Action: ..."
            lines.append(step["action"])
        if step.get("observation"):
            lines.append(f"Observation: {step['observation']}\n")

    lines.append("Next step:")

    # No trailing "Thought:" line here!
    return "\n".join(lines)


In [ ]:
# milestone 3: language model (hugging face)

In [ ]:
# language_model.py

import torch
from transformers import AutoModelForCausalLM, AutoTokenizer


class LLMWrapper:
    def __init__(
        self,
        model_name="Qwen/Qwen2.5-1.5B-Instruct",
        max_new_tokens=120,
        temperature=0.2,
        top_p=0.95
    ):
        """
        Loads a small GPT-style model from Hugging Face.
        Designed to output ReAct steps:
            Thought: ...
            Action: search[...] or Action: finish[...]
        """
        self.device = "cuda" if torch.cuda.is_available() else "cpu"

        print(f"Loading model: {model_name} on {self.device} ...")
        self.tokenizer = AutoTokenizer.from_pretrained(model_name)
        self.model = AutoModelForCausalLM.from_pretrained(model_name).to(self.device)

        self.max_new_tokens = max_new_tokens
        self.temperature = temperature
        self.top_p = top_p

    def __call__(self, prompt: str) -> str:
      # Tokenize
      enc = self.tokenizer(prompt, return_tensors="pt")
      input_ids = enc.input_ids.to(self.device)

    # Build a simple attention mask to avoid the warning
      attention_mask = (input_ids != self.tokenizer.eos_token_id).to(self.device)

    # Use deterministic decoding to reduce format drift
      output = self.model.generate(
          input_ids,
          attention_mask=attention_mask,
          max_new_tokens=self.max_new_tokens,
          do_sample=False,          # turn off sampling
          temperature=0.0,          # ignored when do_sample=False, but explicit
          top_p=1.0,
          pad_token_id=self.tokenizer.eos_token_id,
      )

      decoded = self.tokenizer.decode(output[0], skip_special_tokens=True)
      return decoded[len(prompt):].strip()


In [ ]:
# Create a simple functional interface LLM(prompt)
_model_instance = None

def LLM(prompt: str) -> str:
    global _model_instance
    if _model_instance is None:
        _model_instance = LLMWrapper()
    return _model_instance(prompt)

In [ ]:
#missing the example he gave in github, not sure how to add it

In [ ]:
from dataclasses import dataclass, asdict
from typing import Callable, Dict, List, Tuple, Optional, Any
import json, re

@dataclass
class Step:
    thought: str
    action: str
    observation: str

@dataclass
class AgentConfig:
    max_steps: int = 6
    allow_tools: Tuple[str, ...] = ("search",)
    verbose: bool = True

class ReActAgent:
    def __init__(self, llm: Callable[[str], str], tools: Dict[str, Dict[str, Any]], config: AgentConfig | None=None):
        self.llm = llm
        self.tools = tools
        self.config = config or AgentConfig()
        self.trajectory: List[Step] = []

    def run(self, user_query: str) -> Dict[str, Any]:
        self.trajectory.clear()

        for step_idx in range(self.config.max_steps):

            # 1. Build prompt
            prompt = make_prompt(user_query, self.trajectory)

            # 2. Model inference
            out = self.llm(prompt)

            # Extract Thought (first one)
            t_match = re.search(r"Thought:\s*(.*)", out)
            thought = t_match.group(1).strip() if t_match else "(no thought)"

# Extract only the core action: search[...] or finish[...]
            a_match = re.search(r"Action:\s*(search\[.*?\]|finish\[.*?\])", out, re.DOTALL)

            if a_match:
              raw_action = a_match.group(1).strip()   # exactly search[...] or finish[...]
              action_line = f"Action: {raw_action}"
            else:
              action_line = 'Action: finish[answer="(no action)"]'


            # 3. Parse action
            parsed = parse_action(action_line)

            if not parsed:
                observation = "Invalid action format. Stopping."
                self.trajectory.append(Step(thought, action_line, observation))
                break

            name = parsed["type"]
            args = parsed["args"]

            if name == "finish":
                observation = "done"
                self.trajectory.append(Step(thought, action_line, observation))
                break

            if name not in self.config.allow_tools or name not in self.tools:
                observation = f"Action '{name}' not allowed."
                self.trajectory.append(Step(thought, action_line, observation))
                break

            try:
                obs_payload = self.tools[name]["fn"](**args)
                observation = json.dumps(obs_payload, ensure_ascii=False)
            except Exception as e:
                observation = f"Tool error: {e}"

            self.trajectory.append(Step(thought, action_line, observation))

        final_answer = None
        for s in reversed(self.trajectory):
            if s.action.startswith("Action: finish["):
                m = re.search(r'answer="(.*)"', s.action)
                if m:
                    final_answer = m.group(1)
                    break

        return {
            "question": user_query,
            "final_answer": final_answer,
            "steps": [asdict(s) for s in self.trajectory]
        }


In [ ]:
from typing import Callable, Dict, List, Tuple, Optional, Any

agent = ReActAgent(LLM, TOOLS, AgentConfig(max_steps=6, verbose=True))

demo_q = "SATAN 2: Russia unvelis an image of its terrifying rocket"
result = agent.run(demo_q)

print("Question:", result["question"])
print("\nFinal Answer:", result["final_answer"])
print("\nTrajectory:")
for i, s in enumerate(result["steps"], 1):
    print(f"\nStep {i}")
    print("Thought:", s["thought"])
    print("Action:", s["action"])
    print("Observation:", s["observation"][:500] + ("..." if len(s["observation"])>500 else ""))

# ----------------------------
# Tiny Evaluation Harness
# ----------------------------
GOLD = {
    "SATAN 2: Russia unvelis an image of its terrifying rocket":
        ["true"],
    "Did Obama ban Christmas trees":
        ["fake"],
    "The Oxford 2025 Word of the Year Is 'Rage Bait'":
        ["true"]
}

def normalize(s: str) -> str:
    return (s or "").lower()

def em_contains(pred: Optional[str], gold_phrases: List[str]) -> bool:
    p = normalize(pred)
    return any(any(g in p for g in (normalize(gp),)) for gp in gold_phrases)

def evaluate(agent: ReActAgent, questions: List[str]) -> Dict[str, Any]:
    rows = []
    for q in questions:
        out = agent.run(q)
        pred = out["final_answer"]
        gold_phrases = GOLD.get(q, [])
        ok = em_contains(pred, gold_phrases) if gold_phrases else (pred is not None)
        rows.append({"question": q, "pred": pred, "ok": ok})
    acc = sum(r["ok"] for r in rows) / max(1, len(rows))
    return {"accuracy": acc, "rows": rows}

eval_out = evaluate(agent, list(GOLD.keys()))
print("\n\nEvaluation (toy):", eval_out["accuracy"])
for r in eval_out["rows"]:
    print("-", r)


Loading model: Qwen/Qwen2.5-1.5B-Instruct on cuda ...
Question: SATAN 2: Russia unvelis an image of its terrifying rocket

Final Answer: (no action)

Trajectory:

Step 1
Thought: I need to find information on whether this claim about Satan 2 being related to Russia's missile program is true or not. It seems like there might be some confusion with the name 'Satan 2', which could refer to different things depending on context. However, if we assume that 'Satan 2' refers to a specific missile system, then searching for information about it would help clarify the situation. Action: search[query="satan 2 russia missile", k=5
Action: Action: search[query="satan 2 russia missile", k=5]
Observation: {"tool": "search", "query": "satan 2 russia missile", "confidence": 0.45877358004305635, "summary": "satan 2 russia missile.", "results": [{"id": "13818", "title": "Why did Satan-2 shock the West?", "score": 0.45877358004305635, "snippet": "Why did Satan-2 shock the West? 31.10.2016 The recent publ

In [ ]:
import numpy as np
import pandas as pd
from tqdm.auto import tqdm
from sklearn.metrics import confusion_matrix, classification_report

# In this dataset: label 1 = fake news, 0 = true news
def pred_to_label(pred: str) -> int | None:
    """
    Map the agent's final_answer string to numeric label.
    Returns:
        0 for 'fake', 1 for 'true', None if can't interpret.
    """
    if pred is None:
        return None
    p = str(pred).lower()
    if "fake" in p:
        return 0
    if "true" in p:
        return 1
    return None

# Sample up to 100 observations from the dataframe
N_SAMPLE = 100
sample_df = df.sample(n=min(N_SAMPLE, len(df)), random_state=7)

gold_labels = []
pred_labels = []
questions = []
raw_answers = []

print(f"Running agent on {len(sample_df)} headlines...")
for _, row in tqdm(sample_df.iterrows(), total=len(sample_df)):

    # Use title as the "headline question" fall back to content if needed
    if pd.notna(row.get("title")) and str(row["title"]).strip():
        q = str(row["title"])
    else:
        q = str(row.get("content", ""))

    out = agent.run(q)
    pred_str = out.get("final_answer", None)

    pred_lbl = pred_to_label(pred_str)  # 1, 0, or None

    questions.append(q)
    raw_answers.append(pred_str)
    gold_labels.append(int(row["label"]))
    pred_labels.append(pred_lbl)

gold_labels = np.array(gold_labels)
pred_labels = np.array(pred_labels, dtype=object)

# How many predictions were usable (true/fake) vs None?
valid_mask = np.array([p in (0, 1) for p in pred_labels])
n_valid = valid_mask.sum()
n_total = len(pred_labels)
print(f"\nValid predictions (true/fake): {n_valid}/{n_total}")
print(f"Invalid / unparsed predictions: {n_total - n_valid}")

if n_valid == 0:
    print("No valid predictions to evaluate. Check the agent's output format.")
else:
    # For evaluation, ignore rows where the model gave no interpretable label
    y_true = gold_labels[valid_mask]
    y_pred = pred_labels[valid_mask].astype(int)

    # Overall accuracy on the valid subset
    acc = (y_true == y_pred).mean()
    print(f"\nAccuracy on {n_valid} valid predictions: {acc:.4f}")

    # Confusion matrix: rows = true (1=true, 0=fake), cols = pred
    cm = confusion_matrix(y_true, y_pred, labels=[0, 1])
    print("\nConfusion matrix (rows=true, cols=pred; 0=true, 1=fake):")
    print(cm)

    # detailed classification report
    print("\nClassification report:")
    print(classification_report(y_true, y_pred, target_names=["true", "fake"]))


Running agent on 100 headlines...


  0%|          | 0/100 [00:00<?, ?it/s]


Valid predictions (true/fake): 62/100
Invalid / unparsed predictions: 38

Accuracy on 62 valid predictions: 0.4677

Confusion matrix (rows=true, cols=pred; 0=true, 1=fake):
[[21  7]
 [26  8]]

Classification report:
              precision    recall  f1-score   support

        true       0.45      0.75      0.56        28
        fake       0.53      0.24      0.33        34

    accuracy                           0.47        62
   macro avg       0.49      0.49      0.44        62
weighted avg       0.49      0.47      0.43        62

